# Mlynatom experiments

TODO add seasonal

In [185]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from scipy import optimize

SEED = 333

In [269]:
def transform_df(df: pd.DataFrame, encoder: OneHotEncoder) -> pd.DataFrame:
    # select all object columns
    obj_cols = df.select_dtypes('object')
    # apply transformation by encoder fitted on the training split
    transformed_cols = encoder.transform(obj_cols)
    # get new feature names
    feature_names = encoder.get_feature_names_out()
    # create new dataframe
    transformed_df = pd.DataFrame(
        transformed_cols, index=df.index, columns=feature_names).astype(bool)
    new_df = pd.concat(
        [df.select_dtypes(exclude='object'), transformed_df], axis=1)
    return new_df

def eval_tscv(tscv: TimeSeriesSplit, alpha: float, X, y, weights, verbose: bool = True):
    rmses = []
    maes = []
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        if isinstance(X, pd.DataFrame):
            X_train = scaler.fit_transform(X.iloc[train_index, :])
            X_test = scaler.transform(X.iloc[test_index, :])
        else:
            X_train = scaler.fit_transform(X[train_index, :])
            X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]
        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)
        y_pred = tmp_ridge.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mae = mean_absolute_error(y_test, y_pred)
        if verbose:
            print("pred", tmp_ridge.predict(X_test),
                  "test:", y_test, "rmse", rmse, "mae", mae)
            print(tmp_ridge.coef_)
            print(tmp_ridge.intercept_)
            print()
        rmses.append(rmse)
        maes.append(mae)

    weighted_rmse = np.average(rmses, weights=weights)
    weighted_mae = np.average(maes, weights=weights)
    print(weighted_rmse)
    print(weighted_mae)

## Prepare Data

In [2]:
orig_df = pd.read_csv('../dataset/team_A_dataset.csv')
orig_df.head

<bound method NDFrame.head of       month  year kraj  general_thefts  break_in_thefts  avg_monthly_salary  \
0         1  2009  HKK           271.0            174.0             19132.0   
1         1  2009  JHC           275.0            181.0             19576.0   
2         1  2009  JHM           909.0            286.0             21065.0   
3         1  2009  KVK           198.0             96.0             18652.0   
4         1  2009  LBK           343.0            130.0             19653.0   
...     ...   ...  ...             ...              ...                 ...   
2473      9  2023  PLK           145.0            130.0             41220.0   
2474      9  2023  STC           262.0            254.0             42990.0   
2475      9  2023  ULK           243.0            200.0             39664.0   
2476      9  2023  VYS            62.0             39.0             39315.0   
2477      9  2023  ZLK            67.0             57.0             38596.0   

      celkem  m_do_65

In [3]:
display(orig_df.isna().sum())

month                               0
year                                0
kraj                                0
general_thefts                      0
break_in_thefts                     0
                                   ..
noveHlasenaAUvolnenaVPM             0
obsazenaAZrusenaVPM                 0
absolventiSkolAMladistvi            0
uchazeciOZamestnaniUoZMuzi_ratio    0
uchazeciOZamestnaniUoZZeny_ratio    0
Length: 117, dtype: int64

In [4]:
orig_df.describe()

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,celkem,m_do_65,z_do_65,celkem_w2,celkem_w3,...,avg_natural_gas_price,uchazeciOZamestnaniUoZ,uchazeciOZamestnaniUoZZeny,uchazeciOZamestnaniUoZMuzi,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi,uchazeciOZamestnaniUoZMuzi_ratio,uchazeciOZamestnaniUoZZeny_ratio
count,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,266.000000,266.000000,266.000000,266.000000,266.000000,...,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000
mean,6.423729,2015.881356,489.349475,233.924939,28453.470944,1394.827068,347.500000,618.560150,2762.872180,4105.748120,...,3.499783,27967.878531,14174.182809,13793.695722,3371.029459,1975.188458,1899.223164,1480.798628,0.486333,0.513667
std,3.429731,4.259787,681.754931,214.294818,6730.770886,5885.779007,785.677495,2600.293961,8979.240558,11466.193698,...,1.236720,17887.574100,8725.674030,9234.474479,1770.417610,1716.588043,1706.252205,1062.353914,0.030833,0.030833
min,1.000000,2009.000000,19.000000,19.000000,17704.000000,-26447.000000,-3008.000000,-12127.000000,-24614.000000,-22643.000000,...,1.714700,5329.000000,2808.000000,2413.000000,817.000000,0.000000,0.000000,165.000000,0.408138,0.431466
25%,3.000000,2012.000000,139.000000,80.000000,22815.000000,276.250000,104.250000,125.250000,543.250000,801.250000,...,2.664000,13350.250000,6925.750000,6407.500000,1965.000000,883.250000,893.250000,633.000000,0.464473,0.491328
50%,6.000000,2016.000000,252.000000,162.000000,26431.500000,509.500000,177.500000,247.500000,1072.500000,1629.000000,...,3.214300,23996.000000,12384.500000,11521.000000,2892.000000,1529.000000,1450.000000,1244.000000,0.486783,0.513217
75%,9.000000,2020.000000,479.000000,282.000000,33427.000000,1317.000000,446.500000,660.000000,3221.250000,5672.750000,...,4.080000,34709.000000,17910.500000,17183.500000,4402.500000,2536.250000,2360.000000,1895.000000,0.508672,0.535527
max,12.000000,2023.000000,4599.000000,1107.000000,53070.000000,60636.000000,6854.000000,27163.000000,74113.000000,83545.000000,...,8.770500,93714.000000,43201.000000,50572.000000,12418.000000,12114.000000,17285.000000,6058.000000,0.568534,0.591862


In [5]:
orig_df.columns

Index(['month', 'year', 'kraj', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'celkem', 'm_do_65', 'z_do_65', 'celkem_w2',
       ...
       'avg_natural_gas_price', 'uchazeciOZamestnaniUoZ',
       'uchazeciOZamestnaniUoZZeny', 'uchazeciOZamestnaniUoZMuzi',
       'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi', 'uchazeciOZamestnaniUoZMuzi_ratio',
       'uchazeciOZamestnaniUoZZeny_ratio'],
      dtype='object', length=117)

### Dataset version 1 - predicting uchazeciOZamestnaniUoZZeny_ratio, all rows, without refugee info -> big (economic) model

In [35]:
drop_cols = [f"celkem_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]


In [36]:
df1 = orig_df.drop(columns=drop_cols)
display(df1.columns)
display(df1.info())

Index(['month', 'year', 'kraj', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'bilance',
       'avg_energy_price', 'avg_gasoline_price', 'avg_natural_gas_price',
       'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi', 'uchazeciOZamestnaniUoZZeny_ratio'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   month                                 2478 non-null   int64  
 1   year                                  2478 non-null   int64  
 2   kraj                                  2478 non-null   object 
 3   general_thefts                        2478 non-null   float64
 4   break_in_thefts                       2478 non-null   float64
 5   avg_monthly_salary                    2478 non-null   float64
 6   monthly_min_wage                      2478 non-null   int64  
 7   monthly_inflation_rate_wrt_last_year  2478 non-null   float64
 8   reer                                  2478 non-null   float64
 9   bilance                               2478 non-null   float64
 10  avg_energy_price                      2478 non-null   float64
 11  avg_gasoline_pric

None

In [37]:
df1

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi,uchazeciOZamestnaniUoZZeny_ratio
0,1,2009,HKK,271.0,174.0,19132.0,8000,2.2,89.45,-2.730,70.209,1.1317,5.0707,4781,1417,2948,1221,0.489849
1,1,2009,JHC,275.0,181.0,19576.0,8000,2.2,89.45,-2.730,70.209,1.1317,5.0707,5538,1053,2289,1512,0.502256
2,1,2009,JHM,909.0,286.0,21065.0,8000,2.2,89.45,-2.730,70.209,1.1317,5.0707,9824,3140,4985,3078,0.505306
3,1,2009,KVK,198.0,96.0,18652.0,8000,2.2,89.45,-2.730,70.209,1.1317,5.0707,3325,496,1050,960,0.479642
4,1,2009,LBK,343.0,130.0,19653.0,8000,2.2,89.45,-2.730,70.209,1.1317,5.0707,4814,1134,1665,1126,0.496367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,11.874,138.980,1.7623,2.7372,2360,1473,1221,809,0.577460
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,11.874,138.980,1.7623,2.7372,5929,2502,1777,2149,0.578063
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,11.874,138.980,1.7623,2.7372,4871,1380,1217,2084,0.574391
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,11.874,138.980,1.7623,2.7372,2006,885,865,769,0.563609


categorical column is kraj (month and year are probably not - in terms of time series)

In [41]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df1.select_dtypes('object')
encoder.fit(obj_cols)

df1 = transform_df(df1, encoder=encoder)

In [42]:
df1

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,bilance,avg_energy_price,...,kraj_LBK,kraj_MSK,kraj_OLK,kraj_PAK,kraj_PHA,kraj_PLK,kraj_STC,kraj_ULK,kraj_VYS,kraj_ZLK
0,1,2009,271.0,174.0,19132.0,8000,2.2,89.45,-2.730,70.209,...,False,False,False,False,False,False,False,False,False,False
1,1,2009,275.0,181.0,19576.0,8000,2.2,89.45,-2.730,70.209,...,False,False,False,False,False,False,False,False,False,False
2,1,2009,909.0,286.0,21065.0,8000,2.2,89.45,-2.730,70.209,...,False,False,False,False,False,False,False,False,False,False
3,1,2009,198.0,96.0,18652.0,8000,2.2,89.45,-2.730,70.209,...,False,False,False,False,False,False,False,False,False,False
4,1,2009,343.0,130.0,19653.0,8000,2.2,89.45,-2.730,70.209,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,145.0,130.0,41220.0,17300,6.9,115.41,11.874,138.980,...,False,False,False,False,False,True,False,False,False,False
2474,9,2023,262.0,254.0,42990.0,17300,6.9,115.41,11.874,138.980,...,False,False,False,False,False,False,True,False,False,False
2475,9,2023,243.0,200.0,39664.0,17300,6.9,115.41,11.874,138.980,...,False,False,False,False,False,False,False,True,False,False
2476,9,2023,62.0,39.0,39315.0,17300,6.9,115.41,11.874,138.980,...,False,False,False,False,False,False,False,False,True,False


In [43]:
df1.isna().sum()

month                                   0
year                                    0
general_thefts                          0
break_in_thefts                         0
avg_monthly_salary                      0
monthly_min_wage                        0
monthly_inflation_rate_wrt_last_year    0
reer                                    0
bilance                                 0
avg_energy_price                        0
avg_gasoline_price                      0
avg_natural_gas_price                   0
noveHlaseniUchazeci                     0
noveHlasenaAUvolnenaVPM                 0
obsazenaAZrusenaVPM                     0
absolventiSkolAMladistvi                0
uchazeciOZamestnaniUoZZeny_ratio        0
kraj_JHC                                0
kraj_JHM                                0
kraj_KVK                                0
kraj_LBK                                0
kraj_MSK                                0
kraj_OLK                                0
kraj_PAK                          

### Dataset version 2 - predicting uchazeciOZamestnaniUoZZeny_ratio, refugees rows -> refugee model

In [44]:
drop_cols2 = [f"celkem_w{w}" for w in range(2,20)]
drop_cols2 += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols2 += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols2 += ["celkem", "m_do_65", "z_do_65", "uchazeciOZamestnaniUoZ", "uchazeciOZamestnaniUoZZeny", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]

In [51]:
df2 = orig_df.drop(columns=drop_cols2)

In [52]:
df2 = df2[(df2.year > 2022) | ((df2.year == 2022) & (df2.month > 2))]
df2

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,...,z_do_65_w19_ratio,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi,uchazeciOZamestnaniUoZZeny_ratio
2212,3,2022,HKK,60.0,53.0,34689.0,16200,12.7,107.78,0.117205,...,0.417727,-12.741,187.10,1.9464,4.9772,1451,2010,2094,527,0.507011
2213,3,2022,JHC,113.0,71.0,33998.0,16200,12.7,107.78,0.123191,...,0.430853,-12.741,187.10,1.9464,4.9772,1616,3249,4076,521,0.520700
2214,3,2022,JHM,224.0,327.0,37027.0,16200,12.7,107.78,0.129461,...,0.426214,-12.741,187.10,1.9464,4.9772,3383,6761,6142,1624,0.511907
2215,3,2022,KVK,54.0,63.0,32424.0,16200,12.7,107.78,0.117503,...,0.437780,-12.741,187.10,1.9464,4.9772,1027,2096,1588,372,0.532359
2216,3,2022,LBK,129.0,116.0,33745.0,16200,12.7,107.78,0.095375,...,0.436593,-12.741,187.10,1.9464,4.9772,1562,3192,3957,566,0.532358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,...,0.489928,11.874,138.98,1.7623,2.7372,2360,1473,1221,809,0.577460
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,...,0.433166,11.874,138.98,1.7623,2.7372,5929,2502,1777,2149,0.578063
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,...,0.408651,11.874,138.98,1.7623,2.7372,4871,1380,1217,2084,0.574391
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,...,0.424424,11.874,138.98,1.7623,2.7372,2006,885,865,769,0.563609


In [53]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df2.select_dtypes('object')
encoder.fit(obj_cols)

df2 = transform_df(df2, encoder=encoder)

In [95]:
display(df2)

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,z_do_65_ratio,...,kraj_LBK,kraj_MSK,kraj_OLK,kraj_PAK,kraj_PHA,kraj_PLK,kraj_STC,kraj_ULK,kraj_VYS,kraj_ZLK
2212,3,2022,60.0,53.0,34689.0,16200,12.7,107.78,0.117205,0.417727,...,False,False,False,False,False,False,False,False,False,False
2213,3,2022,113.0,71.0,33998.0,16200,12.7,107.78,0.123191,0.430853,...,False,False,False,False,False,False,False,False,False,False
2214,3,2022,224.0,327.0,37027.0,16200,12.7,107.78,0.129461,0.426214,...,False,False,False,False,False,False,False,False,False,False
2215,3,2022,54.0,63.0,32424.0,16200,12.7,107.78,0.117503,0.437780,...,False,False,False,False,False,False,False,False,False,False
2216,3,2022,129.0,116.0,33745.0,16200,12.7,107.78,0.095375,0.436593,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,0.549075,...,False,False,False,False,False,True,False,False,False,False
2474,9,2023,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,0.509645,...,False,False,False,False,False,False,True,False,False,False
2475,9,2023,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,0.370968,...,False,False,False,False,False,False,False,True,False,False
2476,9,2023,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,0.488024,...,False,False,False,False,False,False,False,False,True,False


### Dataset version 1.2 predicting uchazeciOZamestnaniUoZZeny, all rows, without refugee info -> big (economic) model

In [190]:
drop_cols = [f"celkem_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]

In [191]:
df12 = orig_df.drop(columns=drop_cols)
display(df12.columns)
display(df12.info())

Index(['month', 'year', 'kraj', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'bilance',
       'avg_energy_price', 'avg_gasoline_price', 'avg_natural_gas_price',
       'uchazeciOZamestnaniUoZZeny', 'noveHlaseniUchazeci',
       'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   month                                 2478 non-null   int64  
 1   year                                  2478 non-null   int64  
 2   kraj                                  2478 non-null   object 
 3   general_thefts                        2478 non-null   float64
 4   break_in_thefts                       2478 non-null   float64
 5   avg_monthly_salary                    2478 non-null   float64
 6   monthly_min_wage                      2478 non-null   int64  
 7   monthly_inflation_rate_wrt_last_year  2478 non-null   float64
 8   reer                                  2478 non-null   float64
 9   bilance                               2478 non-null   float64
 10  avg_energy_price                      2478 non-null   float64
 11  avg_gasoline_pric

None

In [192]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df12.select_dtypes('object')
encoder.fit(obj_cols)

df12 = transform_df(df12, encoder=encoder)

### Dataset version 2.2 - predicting uchazeciOZamestnaniUoZZeny, refugees rows -> refugee model

In [200]:
drop_cols2 = [f"celkem_w{w}" for w in range(2,20)]
drop_cols2 += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols2 += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols2 += ["celkem", "m_do_65", "z_do_65", "uchazeciOZamestnaniUoZ", "uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]

In [201]:
df22 = orig_df.drop(columns=drop_cols2)
df22 = df22[(df22.year > 2022) | ((df22.year == 2022) & (df22.month > 2))]
df22

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,...,z_do_65_w19_ratio,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,uchazeciOZamestnaniUoZZeny,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi
2212,3,2022,HKK,60.0,53.0,34689.0,16200,12.7,107.78,0.117205,...,0.417727,-12.741,187.10,1.9464,4.9772,5315,1451,2010,2094,527
2213,3,2022,JHC,113.0,71.0,33998.0,16200,12.7,107.78,0.123191,...,0.430853,-12.741,187.10,1.9464,4.9772,6163,1616,3249,4076,521
2214,3,2022,JHM,224.0,327.0,37027.0,16200,12.7,107.78,0.129461,...,0.426214,-12.741,187.10,1.9464,4.9772,16681,3383,6761,6142,1624
2215,3,2022,KVK,54.0,63.0,32424.0,16200,12.7,107.78,0.117503,...,0.437780,-12.741,187.10,1.9464,4.9772,4409,1027,2096,1588,372
2216,3,2022,LBK,129.0,116.0,33745.0,16200,12.7,107.78,0.095375,...,0.436593,-12.741,187.10,1.9464,4.9772,5931,1562,3192,3957,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,...,0.489928,11.874,138.98,1.7623,2.7372,6538,2360,1473,1221,809
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,...,0.433166,11.874,138.98,1.7623,2.7372,17887,5929,2502,1777,2149
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,...,0.408651,11.874,138.98,1.7623,2.7372,17728,4871,1380,1217,2084
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,...,0.424424,11.874,138.98,1.7623,2.7372,5303,2006,885,865,769


In [202]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df22.select_dtypes('object')
encoder.fit(obj_cols)

df22 = transform_df(df22, encoder=encoder)

In [203]:
display(df22)

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,z_do_65_ratio,...,kraj_LBK,kraj_MSK,kraj_OLK,kraj_PAK,kraj_PHA,kraj_PLK,kraj_STC,kraj_ULK,kraj_VYS,kraj_ZLK
2212,3,2022,60.0,53.0,34689.0,16200,12.7,107.78,0.117205,0.417727,...,False,False,False,False,False,False,False,False,False,False
2213,3,2022,113.0,71.0,33998.0,16200,12.7,107.78,0.123191,0.430853,...,False,False,False,False,False,False,False,False,False,False
2214,3,2022,224.0,327.0,37027.0,16200,12.7,107.78,0.129461,0.426214,...,False,False,False,False,False,False,False,False,False,False
2215,3,2022,54.0,63.0,32424.0,16200,12.7,107.78,0.117503,0.437780,...,False,False,False,False,False,False,False,False,False,False
2216,3,2022,129.0,116.0,33745.0,16200,12.7,107.78,0.095375,0.436593,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,0.549075,...,False,False,False,False,False,True,False,False,False,False
2474,9,2023,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,0.509645,...,False,False,False,False,False,False,True,False,False,False
2475,9,2023,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,0.370968,...,False,False,False,False,False,False,False,True,False,False
2476,9,2023,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,0.488024,...,False,False,False,False,False,False,False,False,True,False


### Dataset version 3 - predicting uchazeciOZamestnaniUoZZeny, all rows, imputation on refugee rows in years before -> refugee model

In [298]:
drop_cols2 = [f"celkem_w{w}" for w in range(2,20)]
drop_cols2 += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols2 += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols2 += ["celkem", "m_do_65", "z_do_65", "uchazeciOZamestnaniUoZ", "uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]

In [299]:
df3 = orig_df.drop(columns=drop_cols2)
df3

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,...,z_do_65_w19_ratio,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,uchazeciOZamestnaniUoZZeny,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi
0,1,2009,HKK,271.0,174.0,19132.0,8000,2.2,89.45,NaN,...,NaN,-2.730,70.209,1.1317,5.0707,8445,4781,1417,2948,1221
1,1,2009,JHC,275.0,181.0,19576.0,8000,2.2,89.45,NaN,...,NaN,-2.730,70.209,1.1317,5.0707,10352,5538,1053,2289,1512
2,1,2009,JHM,909.0,286.0,21065.0,8000,2.2,89.45,NaN,...,NaN,-2.730,70.209,1.1317,5.0707,24333,9824,3140,4985,3078
3,1,2009,KVK,198.0,96.0,18652.0,8000,2.2,89.45,NaN,...,NaN,-2.730,70.209,1.1317,5.0707,7386,3325,496,1050,960
4,1,2009,LBK,343.0,130.0,19653.0,8000,2.2,89.45,NaN,...,NaN,-2.730,70.209,1.1317,5.0707,9563,4814,1134,1665,1126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,...,0.489928,11.874,138.980,1.7623,2.7372,6538,2360,1473,1221,809
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,...,0.433166,11.874,138.980,1.7623,2.7372,17887,5929,2502,1777,2149
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,...,0.408651,11.874,138.980,1.7623,2.7372,17728,4871,1380,1217,2084
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,...,0.424424,11.874,138.980,1.7623,2.7372,5303,2006,885,865,769


In [301]:
df3 = df3.fillna(0)
df3

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,m_do_65_ratio,...,z_do_65_w19_ratio,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,uchazeciOZamestnaniUoZZeny,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi
0,1,2009,HKK,271.0,174.0,19132.0,8000,2.2,89.45,0.000000,...,0.000000,-2.730,70.209,1.1317,5.0707,8445,4781,1417,2948,1221
1,1,2009,JHC,275.0,181.0,19576.0,8000,2.2,89.45,0.000000,...,0.000000,-2.730,70.209,1.1317,5.0707,10352,5538,1053,2289,1512
2,1,2009,JHM,909.0,286.0,21065.0,8000,2.2,89.45,0.000000,...,0.000000,-2.730,70.209,1.1317,5.0707,24333,9824,3140,4985,3078
3,1,2009,KVK,198.0,96.0,18652.0,8000,2.2,89.45,0.000000,...,0.000000,-2.730,70.209,1.1317,5.0707,7386,3325,496,1050,960
4,1,2009,LBK,343.0,130.0,19653.0,8000,2.2,89.45,0.000000,...,0.000000,-2.730,70.209,1.1317,5.0707,9563,4814,1134,1665,1126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,0.277383,...,0.489928,11.874,138.980,1.7623,2.7372,6538,2360,1473,1221,809
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,0.338071,...,0.433166,11.874,138.980,1.7623,2.7372,17887,5929,2502,1777,2149
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,0.430108,...,0.408651,11.874,138.980,1.7623,2.7372,17728,4871,1380,1217,2084
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,0.350299,...,0.424424,11.874,138.980,1.7623,2.7372,5303,2006,885,865,769


In [302]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df3.select_dtypes('object')
encoder.fit(obj_cols)

df3 = transform_df(df3, encoder=encoder)

### Dataset version 4 - predicting uchazeciOZamestnaniUoZZeny, refugee rows -> small economic model

In [319]:
drop_cols = [f"celkem_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny_ratio", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]

In [320]:
df4 = orig_df.drop(columns=drop_cols)
df4 = df4[(df4.year > 2022) | ((df4.year == 2022) & (df4.month > 2))]
df4

,month,year,kraj,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,uchazeciOZamestnaniUoZZeny,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi
2212,3,2022,HKK,60.0,53.0,34689.0,16200,12.7,107.78,-12.741,187.10,1.9464,4.9772,5315,1451,2010,2094,527
2213,3,2022,JHC,113.0,71.0,33998.0,16200,12.7,107.78,-12.741,187.10,1.9464,4.9772,6163,1616,3249,4076,521
2214,3,2022,JHM,224.0,327.0,37027.0,16200,12.7,107.78,-12.741,187.10,1.9464,4.9772,16681,3383,6761,6142,1624
2215,3,2022,KVK,54.0,63.0,32424.0,16200,12.7,107.78,-12.741,187.10,1.9464,4.9772,4409,1027,2096,1588,372
2216,3,2022,LBK,129.0,116.0,33745.0,16200,12.7,107.78,-12.741,187.10,1.9464,4.9772,5931,1562,3192,3957,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,9,2023,PLK,145.0,130.0,41220.0,17300,6.9,115.41,11.874,138.98,1.7623,2.7372,6538,2360,1473,1221,809
2474,9,2023,STC,262.0,254.0,42990.0,17300,6.9,115.41,11.874,138.98,1.7623,2.7372,17887,5929,2502,1777,2149
2475,9,2023,ULK,243.0,200.0,39664.0,17300,6.9,115.41,11.874,138.98,1.7623,2.7372,17728,4871,1380,1217,2084
2476,9,2023,VYS,62.0,39.0,39315.0,17300,6.9,115.41,11.874,138.98,1.7623,2.7372,5303,2006,885,865,769


In [321]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df4.select_dtypes('object')
encoder.fit(obj_cols)

df4 = transform_df(df4, encoder=encoder)

## Ridge Regression

### df1

In [97]:
n_splits = len(df2) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny_ratio"
X = df1.drop(columns=y_col_name).to_numpy()
y = df1[y_col_name].to_numpy()

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [98]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
display(res)


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [104]:
rmses = []
maes = []
for train_index, test_index in tscv.split(X):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X[train_index, :])
    X_test = scaler.transform(X[test_index, :])
    y_train = y[train_index]
    y_test = y[test_index]
    tmp_ridge = Ridge(alpha=best_alpha, random_state=SEED)
    tmp_ridge.fit(X_train, y_train)
    y_pred = tmp_ridge.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    print("pred", tmp_ridge.predict(X_test),
          "test:", y_test, "rmse", rmse, "mae", mae)
    rmses.append(rmse)
    maes.append(mae)

weighted_rmse = np.average(rmses, weights=weights)
weighted_mae = np.average(maes, weights=weights)
print(weighted_rmse)
print(weighted_mae)

pred [0.52503995] test: [0.52069956] rmse 0.004340394110931012 mae 0.004340394110931012
pred [0.5162329] test: [0.51190695] rmse 0.004325946848066309 mae 0.004325946848066309
pred [0.49292837] test: [0.53235933] rmse 0.03943095884244707 mae 0.03943095884244707
pred [0.52931933] test: [0.53235796] rmse 0.003038627351448664 mae 0.003038627351448664
pred [0.49777629] test: [0.48715286] rmse 0.010623423629164519 mae 0.010623423629164519
pred [0.50681871] test: [0.50655175] rmse 0.00026696223861355506 mae 0.00026696223861355506
pred [0.51373343] test: [0.51663405] rmse 0.0029006161104331296 mae 0.0029006161104331296
pred [0.50905451] test: [0.53051357] rmse 0.021459057489093714 mae 0.021459057489093714
pred [0.53307562] test: [0.53394841] rmse 0.0008727860771845508 mae 0.0008727860771845508
pred [0.54000142] test: [0.54000142] rmse 2.5654922630735655e-11 mae 2.5654922630735655e-11
pred [0.52075008] test: [0.54799567] rmse 0.027245591060119367 mae 0.027245591060119367
pred [0.51932986] test:

In [78]:
res

 message: Solution found.
 success: True
  status: 0
     fun: [ 5.378e+00]
       x: [ 4.239e+01]
     nit: 33
    nfev: 33

In [146]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sma

X = df1.drop(columns=y_col_name).to_numpy()
X = sma.add_constant(X)
y = df1[y_col_name].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=84, shuffle=False)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [147]:
y_train

array([0.48984919, 0.50225608, 0.50530578, ..., 0.56236532, 0.51740295,
       0.50510294])

In [148]:
X_train

array([[ 0.        , -1.56048665, -1.61150444, ..., -0.2773501 ,
        -0.2773501 , -0.2773501 ],
       [ 0.        , -1.56048665, -1.61150444, ..., -0.2773501 ,
        -0.2773501 , -0.2773501 ],
       [ 0.        , -1.56048665, -1.61150444, ..., -0.2773501 ,
        -0.2773501 , -0.2773501 ],
       ...,
       [ 0.        , -0.98477313,  1.79056049, ...,  3.60555128,
        -0.2773501 , -0.2773501 ],
       [ 0.        , -0.98477313,  1.79056049, ..., -0.2773501 ,
         3.60555128, -0.2773501 ],
       [ 0.        , -0.98477313,  1.79056049, ..., -0.2773501 ,
        -0.2773501 ,  3.60555128]])

In [149]:
best_alpha

31.333883214358604

In [150]:
from sklearn.metrics import r2_score


model = sma.OLS(endog=y_train, exog=X_train)
#fit = model.fit_regularized(alpha=best_alpha, L1_wt= 0)
fit = model.fit()
cmp = y_test, fit.predict(X_test)
mse = mean_squared_error(*cmp)
mae = mean_absolute_error(*cmp)
r2 = r2_score(*cmp)

print(mse, mae, r2)
fit.summary()

0.306420352563224 0.553398020485247 -775.8103348602441


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.002
Model:                            OLS   Adj. R-squared (uncentered):             -0.010
Method:                 Least Squares   F-statistic:                             0.1635
Date:                Tue, 26 Dec 2023   Prob (F-statistic):                        1.00
Time:                        13:29:40   Log-Likelihood:                         -1795.9
No. Observations:                2394   AIC:                                      3650.
Df Residuals:                    2365   BIC:                                      3818.
Df Model:                          29                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.914e-19   2.43e-18     -0.120      0.905   -5.07e-18    4.48e-18
x1             0.0156      0.018      0.862      0.389      -0.020       0.051
x2             0.0144      0.069      0.210      0.834      -0.120       0.149
x3            -0.0020      0.030     -0.066      0.947      -0.061       0.057
x4            -0.0049      0.025     -0.192      0.848      -0.055       0.045
x5            -0.0203      0.085     -0.240      0.811      -0.187       0.146
x6            -0.0035      0.094     -0.038      0.970      -0.187       0.180
x7             0.0084      0.030      0.275      0.783      -0.051       0.068
x8             0.0004      0.045      0.008      0.994      -0.087       0.088
x9            -0.0005      0.011     -0.047      0.963      -0.022       0.021
x10            0.0053      0.030      0.180      0.857      -0.053       0.064
x11            0.0004      0.026      0.016      0.987      -0.051       0.052
x12           -0.0020      0.020     -0.100      0.920      -0.041       0.037
x13           -0.0057      0.027     -0.210      0.833      -0.058       0.047
x14            0.0047      0.028      0.170      0.865      -0.050       0.059
x15            0.0002      0.025      0.006      0.995      -0.048       0.049
x16           -0.0076      0.029     -0.259      0.796      -0.065       0.050
x17            0.0028      0.015      0.182      0.856      -0.027       0.032
x18            0.0049      0.021      0.240      0.810      -0.035       0.045
x19           -0.0061      0.016     -0.374      0.708      -0.038       0.026
x20            0.0044      0.015      0.300      0.764      -0.024       0.033
x21            0.0007      0.022      0.032      0.974      -0.042       0.044
x22           -0.0014      0.016     -0.087      0.931      -0.032       0.029
x23           -0.0015      0.015     -0.094      0.925      -0.032       0.029
x24            0.0140      0.046      0.303      0.762      -0.077       0.105
x25            0.0043      0.016      0.278      0.781      -0.026       0.035
x26            0.0104      0.022      0.479      0.632      -0.032       0.053
x27            0.0065      0.017      0.379      0.705      -0.027       0.040
x28            0.0016      0.015      0.106      0.915      -0.029       0.032
x29           -0.0045      0.016     -0.281      0.779      -0.036       0.027
==============================================================================
Omnibus:                       11.072   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.196
Skew:                          -0.167   Prob(JB):                      0.00371
Kurtosis:                       2.965   Cond. No.            

### df12

In [193]:
n_splits = len(df2) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny"
X = df12.drop(columns=y_col_name).to_numpy()
y = df12[y_col_name].to_numpy()

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [194]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x
display(res)


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


 message: Solution found.
 success: True
  status: 0
     fun: 1489.241882698458
       x: 157.51893148599052
     nit: 34
    nfev: 34

In [198]:
eval_tscv(tscv=tscv, alpha=best_alpha, X, y)

pred [4318.12112924] test: [6163] rmse 1844.8788707649655 mae 1844.8788707649655
[ -354.0232617     55.12018157  -531.05074921   125.72496106
   372.1335161   -543.52013927  -244.7116453  -1370.36712532
   177.41447709   208.12460256   896.10297077  -110.88255759
  1079.77117507    72.35219627  -262.45326212  3591.70905819
  -271.23894394  1656.89260384  -530.95900102  -128.46683345
  2767.3601654    256.33167428  -510.94112719  1238.67338213
  -234.15852396  1458.74858806  1913.29153155  -432.93319705
  -183.85775562]
pred [16650.19766437] test: [16681] rmse 30.802335627558932 mae 30.802335627558932
[ -354.24455149    55.7318608   -529.58565493   124.31901909
   369.78989968  -542.23723453  -239.22923111 -1375.4007729
   177.7284007    209.05276782   897.25457221  -109.09079557
  1078.94815367    71.17327067  -260.82185415  3592.03798755
  -269.1606981   1656.82721364  -530.95008837  -128.44802676
  2767.05765865   256.21499876  -511.00886235  1239.34484479
  -234.00064431  1459.10124

### Ridge Regression df22

In [315]:
n_splits = len(df22) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny"
X = df22.drop(columns=y_col_name)
y = df22[y_col_name].to_numpy()

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [316]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

# do only every 14 -> do not leak data from other kraj in the month


def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X.iloc[train_index, :])
        X_test = scaler.transform(X.iloc[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [317]:
display(res)

 message: Solution found.
 success: True
  status: 0
     fun: 1118.2613374276439
       x: 12.734707504541374
     nit: 36
    nfev: 36

In [318]:
eval_tscv(tscv, best_alpha, X, y, weights=weights)

pred [5315.] test: [6163] rmse 848.0 mae 848.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
5315.0

pred [3761.32327661] test: [16681] rmse 12919.67672339376 mae 12919.67672339376
[ 0.          0.          8.09664743  8.09664743 -8.09664743  0.
  0.          0.          8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743  8.09664743
  8.09664743  8.09664743  8.09664743  8.09664743  0.          0.
  0.          0.          8.09664743  8.09664743  8.09664743 -8.09664743
  8.09664743  0.   

pred [4134.46242037] test: [4416] rmse 281.5375796260196 mae 281.5375796260196
[  99.03382737    0.          554.64860463  739.22662708    7.3763022
    0.           99.03382737   99.03382737    1.44470892 -106.93180463
  -38.3351158   -38.3351158   -38.3351158   -38.3351158   -38.3351158
  -38.3351158   -38.3351158   -38.3351158   -38.3351158   -38.3351158
  -38.3351158   -38.3351158   -38.3351158   -38.3351158   -38.3351158
  -38.3351158   -38.3351158   -38.3351158    -9.07140079   -9.07140079
   -9.07140079   -9.07140079   -9.07140079   -9.07140079   -9.07140079
   -9.07140079   -9.07140079   -9.07140079   -9.07140079   -9.07140079
   -9.07140079   -9.07140079   -9.07140079   -9.07140079   -9.07140079
   -9.07140079  -99.03382737   99.03382737  -99.03382737   99.03382737
 1111.93462703  352.47552588  229.42452908 1200.5183432  -104.73785474
  651.7066989  -179.9610848  -334.8713227   855.0773386  -100.69497476
 -211.69540158 -161.77944823 -162.45830711   19.20045711  410.512007
 -11

In [216]:
df22.columns

Index(['month', 'year', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'm_do_65_ratio',
       'z_do_65_ratio', 'm_do_65_w2_ratio', 'm_do_65_w3_ratio',
       'm_do_65_w4_ratio', 'm_do_65_w5_ratio', 'm_do_65_w6_ratio',
       'm_do_65_w7_ratio', 'm_do_65_w8_ratio', 'm_do_65_w9_ratio',
       'm_do_65_w10_ratio', 'm_do_65_w11_ratio', 'm_do_65_w12_ratio',
       'm_do_65_w13_ratio', 'm_do_65_w14_ratio', 'm_do_65_w15_ratio',
       'm_do_65_w16_ratio', 'm_do_65_w17_ratio', 'm_do_65_w18_ratio',
       'm_do_65_w19_ratio', 'z_do_65_w2_ratio', 'z_do_65_w3_ratio',
       'z_do_65_w4_ratio', 'z_do_65_w5_ratio', 'z_do_65_w6_ratio',
       'z_do_65_w7_ratio', 'z_do_65_w8_ratio', 'z_do_65_w9_ratio',
       'z_do_65_w10_ratio', 'z_do_65_w11_ratio', 'z_do_65_w12_ratio',
       'z_do_65_w13_ratio', 'z_do_65_w14_ratio', 'z_do_65_w15_ratio',
       'z_do_65_w16_ratio', 'z_do_65_w17_ratio', 'z_do_65_w18_ratio'

In [214]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sma

X = df22.drop(columns=y_col_name).to_numpy(dtype=float)
X = sma.add_constant(X)
y = df22[y_col_name].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=84, shuffle=False)

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
X

array([[1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [215]:
from sklearn.metrics import r2_score


#model = sma.OLS(endog=y_train, exog=X_train)
model = sma.RLM(endog=y_train, exog=X_train, M=sma.robust.norms.HuberT())
#fit = model.fit_regularized(alpha=best_alpha, L1_wt= 0)
fit = model.fit()
cmp = y_test, fit.predict(X_test)
mse = mean_squared_error(*cmp)
mae = mean_absolute_error(*cmp)
r2 = r2_score(*cmp)

print(mse, mae, r2)
fit.summary()

754650964.9366258 14904.609519244133 -21.388132825907917


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      y   No. Observations:                  182
Model:                            RLM   Df Residuals:                      127
Method:                          IRLS   Df Model:                           54
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Tue, 26 Dec 2023                                         
Time:                        15:04:43                                         
No. Iterations:                    43                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.004     -0.212      0.832      -0.008       0.007
x1           -45.3331     97.774     -0.464      0.643    -236.966     146.300
x2            -1.5983      7.537     -0.212      0.832     -16.371      13.175
x3             0.5706      0.711      0.803      0.422      -0.822       1.964
x4            -0.0924      0.759     -0.122      0.903      -1.580       1.395
x5            -0.0349      0.038     -0.908      0.364      -0.110       0.040
x6            -0.2526      1.488     -0.170      0.865      -3.169       2.664
x7          -130.5758    127.972     -1.020      0.308    -381.397     120.245
x8           190.6420    171.402      1.112      0.266    -145.300     526.584
x9          -156.3275    343.520     -0.455      0.649    -829.614     516.959
x10          124.5887    460.634      0.270      0.787    -778.238    1027.416
x11          991.7014    620.014      1.599      0.110    -223.505    2206.907
x12         -937.9150   1003.228     -0.935      0.350   -2904.205    1028.375
x13         1239.5997   1206.373      1.028      0.304   -1124.847    3604.047
x14          637.5680   1934.438      0.330      0.742   -3153.862    4428.998
x15          236.2193   2640.443      0.089      0.929   -4938.953    5411.392
x16         2142.3331   2897.147      0.739      0.460   -3535.971    7820.637
x17          612.8560   2236.347      0.274      0.784   -3770.303    4996.015
x18         -657.6154    910.600     -0.722      0.470   -2442.359    1127.129
x19        -3199.5754   2526.610     -1.266      0.205   -8151.641    1752.490
x20         4488.4740   2093.348      2.144      0.032     385.587    8591.361
x21        -1976.0715   2732.095     -0.723      0.470   -7330.879    3378.736
x22        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x23        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x24        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x25        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x26        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x27        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x28        -1044.3700    923.729     -1.131      0.258   -2854.846     766.106
x29        -1117.4512   1251.419     -0.893      0.372   -3570.188    1335.285
x30         2383.7966   1533.999      1.554      0.120    -622.787    5390.380
x31        -1596.0338   1691.541     -0.944      0.345   -4911.394    1719.327
x32         3362.5058   3650.240      0.921      0.357   -3791.833    1.05e+04
x33         -158.6368   5087.470     -0.031      0.975   -1.01e+04    9812.621
x34        -2940.0759   5658.671     -0.520      0.603    -1.4e+04    8150.716
x35         8315.9158   5999.167      1.386      0.166   -3442.234    2.01e+04
x3

In [ ]:
df2.columns

Index(['month', 'year', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'm_do_65_ratio',
       'z_do_65_ratio', 'm_do_65_w2_ratio', 'm_do_65_w3_ratio',
       'm_do_65_w4_ratio', 'm_do_65_w5_ratio', 'm_do_65_w6_ratio',
       'm_do_65_w7_ratio', 'm_do_65_w8_ratio', 'm_do_65_w9_ratio',
       'm_do_65_w10_ratio', 'm_do_65_w11_ratio', 'm_do_65_w12_ratio',
       'm_do_65_w13_ratio', 'm_do_65_w14_ratio', 'm_do_65_w15_ratio',
       'm_do_65_w16_ratio', 'm_do_65_w17_ratio', 'm_do_65_w18_ratio',
       'm_do_65_w19_ratio', 'z_do_65_w2_ratio', 'z_do_65_w3_ratio',
       'z_do_65_w4_ratio', 'z_do_65_w5_ratio', 'z_do_65_w6_ratio',
       'z_do_65_w7_ratio', 'z_do_65_w8_ratio', 'z_do_65_w9_ratio',
       'z_do_65_w10_ratio', 'z_do_65_w11_ratio', 'z_do_65_w12_ratio',
       'z_do_65_w13_ratio', 'z_do_65_w14_ratio', 'z_do_65_w15_ratio',
       'z_do_65_w16_ratio', 'z_do_65_w17_ratio', 'z_do_65_w18_ratio'

### df2

In [158]:
n_splits = len(df2) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny_ratio"
X = df2.drop(columns=y_col_name)
y = df2[y_col_name].to_numpy()

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [169]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

# do only every 14 -> do not leak data from other kraj in the month


def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X.iloc[train_index, :])
        X_test = scaler.transform(X.iloc[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [167]:
display(res)

 message: Solution found.
 success: True
  status: 0
     fun: 0.021607479938802538
       x: 999.999970046681
     nit: 36
    nfev: 36

In [165]:
eval_tscv(tscv, best_alpha, X, y)

pred [0.50701135] test: [0.52069956] rmse 0.01368820895008993 mae 0.01368820895008993
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
pred [0.48674641] test: [0.51190695] rmse 0.025160547459305405 mae 0.025160547459305405
[ 0.          0.          0.00011098  0.00011098 -0.00011098  0.
  0.          0.          0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098  0.00011098
  0.00011098  0.00011098  0.00011098  0.00011098  0.          0.
  0.          0.          0.00011098  0.00011098  0.0

In [182]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sma

X = df2.drop(columns=y_col_name).to_numpy(dtype=float)
X = sma.add_constant(X)
y = df2[y_col_name].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=84, shuffle=False)

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
X

array([[1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 3.000e+00, 2.022e+03, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.000e+00, 9.000e+00, 2.023e+03, ..., 0.000e+00, 0.000e+00,
        1.000e+00]])

In [183]:
from sklearn.metrics import r2_score


#model = sma.OLS(endog=y_train, exog=X_train)
model = sma.RLM(endog=y_train, exog=X_train, M=sma.robust.norms.HuberT())
#fit = model.fit_regularized(alpha=best_alpha, L1_wt= 0)
fit = model.fit()
cmp = y_test, fit.predict(X_test)
mse = mean_squared_error(*cmp)
mae = mean_absolute_error(*cmp)
r2 = r2_score(*cmp)

print(mse, mae, r2)
fit.summary()

2.5723027266328016 0.6737336793644056 -6520.079052754033


<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      y   No. Observations:                  182
Model:                            RLM   Df Residuals:                      127
Method:                          IRLS   Df Model:                           54
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Tue, 26 Dec 2023                                         
Time:                        14:23:39                                         
No. Iterations:                    50                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         2.1e-07   1.29e-07      1.623      0.105   -4.37e-08    4.64e-07
x1            -0.0028      0.003     -0.822      0.411      -0.009       0.004
x2             0.0004      0.000      1.623      0.105   -8.76e-05       0.001
x3          1.625e-05   2.45e-05      0.664      0.507   -3.17e-05    6.43e-05
x4          1.422e-05   2.62e-05      0.544      0.587    -3.7e-05    6.55e-05
x5          2.022e-06   1.32e-06      1.526      0.127   -5.75e-07    4.62e-06
x6          5.545e-06   5.13e-05      0.108      0.914    -9.5e-05       0.000
x7             0.0053      0.004      1.208      0.227      -0.003       0.014
x8            -0.0065      0.006     -1.100      0.271      -0.018       0.005
x9             0.0096      0.012      0.815      0.415      -0.014       0.033
x10           -0.0415      0.016     -2.614      0.009      -0.073      -0.010
x11           -0.0055      0.021     -0.257      0.797      -0.047       0.036
x12            0.0240      0.035      0.695      0.487      -0.044       0.092
x13           -0.0362      0.042     -0.870      0.384      -0.118       0.045
x14           -0.0206      0.067     -0.309      0.757      -0.151       0.110
x15            0.1001      0.091      1.100      0.271      -0.078       0.278
x16            0.1070      0.100      1.072      0.284      -0.089       0.303
x17           -0.1126      0.077     -1.461      0.144      -0.264       0.038
x18            0.0213      0.031      0.679      0.497      -0.040       0.083
x19           -0.1139      0.087     -1.308      0.191      -0.285       0.057
x20            0.1477      0.072      2.047      0.041       0.006       0.289
x21            0.1806      0.094      1.919      0.055      -0.004       0.365
x22            0.0109      0.032      0.342      0.732      -0.051       0.073
x23            0.0109      0.032      0.342      0.732      -0.051       0.073
x24            0.0109      0.032      0.342      0.732      -0.051       0.073
x25            0.0109      0.032      0.342      0.732      -0.051       0.073
x26            0.0109      0.032      0.342      0.732      -0.051       0.073
x27            0.0109      0.032      0.342      0.732      -0.051       0.073
x28            0.0109      0.032      0.342      0.732      -0.051       0.073
x29            0.0336      0.043      0.780      0.435      -0.051       0.118
x30           -0.0147      0.053     -0.278      0.781      -0.118       0.089
x31            0.0306      0.058      0.525      0.600      -0.084       0.145
x32            0.0548      0.126      0.436      0.663      -0.192       0.301
x33           -0.0680      0.175     -0.388      0.698      -0.412       0.276
x34            0.1634      0.195      0.838      0.402      -0.219       0.546
x35            0.0978      0.207      0.473      0.636      -0.307       0.503
x3

In [173]:
df2.columns

Index(['month', 'year', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'm_do_65_ratio',
       'z_do_65_ratio', 'm_do_65_w2_ratio', 'm_do_65_w3_ratio',
       'm_do_65_w4_ratio', 'm_do_65_w5_ratio', 'm_do_65_w6_ratio',
       'm_do_65_w7_ratio', 'm_do_65_w8_ratio', 'm_do_65_w9_ratio',
       'm_do_65_w10_ratio', 'm_do_65_w11_ratio', 'm_do_65_w12_ratio',
       'm_do_65_w13_ratio', 'm_do_65_w14_ratio', 'm_do_65_w15_ratio',
       'm_do_65_w16_ratio', 'm_do_65_w17_ratio', 'm_do_65_w18_ratio',
       'm_do_65_w19_ratio', 'z_do_65_w2_ratio', 'z_do_65_w3_ratio',
       'z_do_65_w4_ratio', 'z_do_65_w5_ratio', 'z_do_65_w6_ratio',
       'z_do_65_w7_ratio', 'z_do_65_w8_ratio', 'z_do_65_w9_ratio',
       'z_do_65_w10_ratio', 'z_do_65_w11_ratio', 'z_do_65_w12_ratio',
       'z_do_65_w13_ratio', 'z_do_65_w14_ratio', 'z_do_65_w15_ratio',
       'z_do_65_w16_ratio', 'z_do_65_w17_ratio', 'z_do_65_w18_ratio'

### Feature selection

In [224]:
from sklearn.feature_selection import SequentialFeatureSelector

In [278]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer


estimator = Ridge()
y_col_name = "uchazeciOZamestnaniUoZZeny"
X = df22.drop(columns=y_col_name)
y = df22[y_col_name].to_numpy()
scaler = StandardScaler()
X_trans = scaler.fit_transform(X)
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)
sfs = SequentialFeatureSelector(estimator=estimator, direction="forward", cv=tscv, scoring=make_scorer(mean_absolute_error))
sfs.fit(X_trans, y)

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=265, test_size=1),
                          estimator=Ridge(),
                          scoring=make_scorer(mean_absolute_error))

In [279]:
mask = sfs.get_support()
mask

array([ True,  True, False, False, False,  True, False,  True, False,
        True, False, False,  True,  True,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [280]:
all_colls = sfs.feature_names_in_
all_colls

AttributeError: 'SequentialFeatureSelector' object has no attribute 'feature_names_in_'

In [281]:
#excluded columns
excluded_cols = all_colls[~mask]
excluded_cols

array(['general_thefts', 'break_in_thefts', 'avg_monthly_salary',
       'monthly_inflation_rate_wrt_last_year', 'm_do_65_ratio',
       'm_do_65_w2_ratio', 'm_do_65_w3_ratio', 'm_do_65_w7_ratio',
       'm_do_65_w8_ratio', 'm_do_65_w18_ratio', 'm_do_65_w19_ratio',
       'z_do_65_w2_ratio', 'z_do_65_w6_ratio', 'z_do_65_w7_ratio',
       'z_do_65_w8_ratio', 'avg_energy_price', 'avg_gasoline_price',
       'avg_natural_gas_price', 'noveHlaseniUchazeci',
       'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi', 'kraj_JHM', 'kraj_KVK', 'kraj_LBK',
       'kraj_MSK', 'kraj_OLK', 'kraj_PAK', 'kraj_PHA', 'kraj_PLK',
       'kraj_STC', 'kraj_ULK', 'kraj_VYS', 'kraj_ZLK'], dtype=object)

In [282]:
features = sfs.get_feature_names_out()
features

array(['x0', 'x1', 'x5', 'x7', 'x9', 'x12', 'x13', 'x14', 'x17', 'x18',
       'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x29', 'x30',
       'x31', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42',
       'x43', 'x44', 'x45', 'x46', 'x54'], dtype=object)

In [284]:
X_select = sfs.transform(X)

/home/mlynatom/SAN_Unemployment_Refugees/.venv/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but SequentialFeatureSelector was fitted without feature names
  warnings.warn(


In [294]:
n_splits = len(df22) - 1
n_splits

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [295]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_select[train_index, :])
        X_test = scaler.transform(X_select[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [296]:
display(res)

 message: Solution found.
 success: True
  status: 0
     fun: 5038.862663229233
       x: 999.9999743212363
     nit: 36
    nfev: 36

In [297]:
eval_tscv(tscv, best_alpha, X_select, y, weights=weights)

pred [5315.] test: [6163] rmse 848.0 mae 848.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
5315.0

pred [5771.55000087] test: [16681] rmse 10909.449999131135 mae 10909.449999131135
[0.         0.         0.         0.         0.80303032 0.80303032
 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032
 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032
 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032
 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032 0.80303032
 0.80303032 0.         0.80303032]
5739.0

pred [9317.5018309] test: [4409] rmse 4908.501830897749 mae 4908.501830897749
[ 0.          0.          0.          0.          3.1757762  13.43693166
 13.43693166 13.43693166 13.43693166 13.43693166 13.43693166 13.43693166
 13.43693166 13.43693166 13.43693166 13.43693166 13.43693166  3.1757762
  3.1757762   3.1757762   3.1757762   3.1757762   3.1757762   3.1757762
  3.1757762   3

decay changes nothing?

### df3

In [306]:
n_splits = len(df2) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny"
X = df3.drop(columns=y_col_name)
y = df3[y_col_name].to_numpy()

q = 1
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [307]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X.iloc[train_index, :])
        X_test = scaler.transform(X.iloc[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [308]:
display(res)

 message: Solution found.
 success: True
  status: 0
     fun: 1090.445700974699
       x: 661.6935209330768
     nit: 27
    nfev: 27

In [309]:
eval_tscv(tscv, best_alpha, X, y, weights=weights)

pred [3656.3229979] test: [6163] rmse 2506.6770021036064 mae 2506.6770021036064
[-2.28481454e+02 -4.28272438e+01 -1.73285408e+02  1.89983516e+02
  1.09051818e+02 -5.17202293e+02 -2.54345566e+02 -9.39798682e+02
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01 -2.36042694e-01
 -2.36042694e-01 -2.36042694e-01  1.51789748e+02  1.17233359e+01
  7.43377260e+02  2.45245387e+00  1.72809197e+03  1.45400619e+02
 -1.31393942e+02  2.91516131e+03 -4.44462207e+02  1.15115088e+03
 -6.334048

### df4

In [326]:
n_splits = len(df2) - 1
n_splits
y_col_name = "uchazeciOZamestnaniUoZZeny"
X = df4.drop(columns=y_col_name)
y = df4[y_col_name].to_numpy()

q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))
weights

array([1.31532113e-06, 1.38454856e-06, 1.45741953e-06, 1.53412582e-06,
       1.61486929e-06, 1.69986241e-06, 1.78932885e-06, 1.88350405e-06,
       1.98263585e-06, 2.08698510e-06, 2.19682642e-06, 2.31244887e-06,
       2.43415670e-06, 2.56227021e-06, 2.69712654e-06, 2.83908057e-06,
       2.98850586e-06, 3.14579564e-06, 3.31136383e-06, 3.48564614e-06,
       3.66910120e-06, 3.86221179e-06, 4.06548609e-06, 4.27945905e-06,
       4.50469373e-06, 4.74178288e-06, 4.99135040e-06, 5.25405305e-06,
       5.53058216e-06, 5.82166543e-06, 6.12806887e-06, 6.45059881e-06,
       6.79010401e-06, 7.14747791e-06, 7.52366096e-06, 7.91964311e-06,
       8.33646643e-06, 8.77522783e-06, 9.23708192e-06, 9.72324413e-06,
       1.02349938e-05, 1.07736777e-05, 1.13407134e-05, 1.19375930e-05,
       1.25658874e-05, 1.32272499e-05, 1.39234209e-05, 1.46562326e-05,
       1.54276132e-05, 1.62395929e-05, 1.70943083e-05, 1.79940087e-05,
       1.89410618e-05, 1.99379598e-05, 2.09873261e-05, 2.20919222e-05,
      

In [327]:
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)

def optimize_alpha(alpha):
    rmses = []
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X.iloc[train_index, :])
        X_test = scaler.transform(X.iloc[test_index, :])
        y_train = y[train_index]
        y_test = y[test_index]

        tmp_ridge = Ridge(alpha=alpha, random_state=SEED)
        tmp_ridge.fit(X_train, y_train)

        rmse = mean_squared_error(
            y_test, tmp_ridge.predict(X_test), squared=False)

        rmses.append(rmse)

    return np.average(rmses, weights=weights)


res = optimize.minimize_scalar(
    optimize_alpha, bounds=(0, 1000), options={"disp": True})
best_alpha = res.x


Optimization terminated successfully;
The returned value satisfies the termination criteria
(using xtol =  1e-05 )


In [328]:
display(res)

 message: Solution found.
 success: True
  status: 0
     fun: 266.74292541986154
       x: 6.638538074770203e-06
     nit: 39
    nfev: 39

In [329]:
eval_tscv(tscv, best_alpha, X, y, weights=weights)

pred [5315.] test: [6163] rmse 848.0 mae 848.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
5315.0

pred [-10618.30449021] test: [16681] rmse 27299.30449021052 mae 27299.30449021052
[  0.           0.          52.99997801  52.99997801 -52.99997801
   0.           0.           0.           0.           0.
   0.           0.          52.99997801  52.99997801  52.99997801
 -52.99997801  52.99997801   0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.        ]
5739.0

pred [3528.6950233] test: [4409] rmse 880.3049767029606 mae 880.3049767029606
[   0.            0.          637.86631343  661.00733499  635.9554334
    0.            0.            0.            0.            0.
    0.            0.          660.87217243  648.18333241  593.80779534
  659.75843035 -297.47995705  659.9430179     0.            0.
    0.            0.            0.            0.            